In [4]:
import numpy as np
import pandas as pd
import re
import json
from utils import load_ollama, prompt_test, run_ollama, process_dataframe_in_chunks
from tqdm import tqdm
tqdm.pandas()

test_csv_path = '../data/test_07.csv'
rag_result_path = '../data/rag_result_05.csv'

test_df = pd.read_csv(test_csv_path, encoding='utf-8-sig')
rag_result_df = pd.read_csv(rag_result_path, encoding='utf-8-sig')

In [ ]:
test_df = test_df.drop(columns=['isvalidllm_answer_1', 'isvalidllm_answer_2', 'isvalidllm_answer_3'])

### 임베딩 벡터 준비

In [21]:
from sentence_transformers import SentenceTransformer

# 모델 불러오기
embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)
answer_cols = sorted([col for col in test_df.columns if 'llm_answer' in col])

def compute_answer_embeddings_for_row(row, answer_cols):
    # 각 행에서 1번부터 25번까지의 문장을 리스트로 모읍니다.
    sentences = [row[col] for col in answer_cols]
    # 25개의 문장에 대해 임베딩을 계산합니다. (반환: (25, embedding_dim))
    embeddings = embedding.encode(sentences)
    return embeddings

# 각 행에 대해 임베딩을 계산하고, Series로 반환된 결과를 ndarray로 스택합니다.
answer_embeddings = test_df.apply(lambda x: compute_answer_embeddings_for_row(x, answer_cols), axis=1)
answer_ndarray = np.stack(answer_embeddings.to_numpy())

print("answer_ndarray shape:", answer_ndarray.shape)

answer_ndarray shape: (964, 3, 768)


In [11]:
rag_gt_cols = sorted([col for col in rag_result_df.columns if 'rerank_gt' in col])

def compute_gt_embeddings_for_row(row, rag_gt_cols):
    # 각 행에서 1번부터 25번까지의 문장을 리스트로 모읍니다.
    sentences = [row[col] for col in rag_gt_cols]
    # 25개의 문장에 대해 임베딩을 계산합니다. (반환: (25, embedding_dim))
    embeddings = embedding.encode(sentences)
    return embeddings

# 각 행에 대해 임베딩을 계산하고, Series로 반환된 결과를 ndarray로 스택합니다.
ground_truth_embeddings = rag_result_df.apply(lambda x: compute_gt_embeddings_for_row(x, rag_gt_cols), axis=1)
ground_truth_ndarray = np.stack(ground_truth_embeddings.to_numpy())

print("ground_truth_ndarray shape:", ground_truth_ndarray.shape)

ground_truth_ndarray shape: (964, 25, 768)


### 코사인 유사도 계산 및 argmax

In [22]:
def best_prediction_index(predictions: np.ndarray, ground_truth: np.ndarray) -> list:
    """
    주어진 예측과 정답 임베딩 벡터들 간의 코사인 유사도의 평균이 가장 높은 예측의 인덱스를 반환합니다.
    
    인자:
        predictions: np.ndarray, shape (N, p, d)
            - N: 데이터 개수
            - p: 각 데이터당 예측 임베딩 개수
            - d: 임베딩 벡터의 차원
        ground_truth: np.ndarray, shape (N, t, d)
            - N: 데이터 개수
            - t: 각 데이터당 정답 임베딩 개수
            - d: 임베딩 벡터의 차원
    
    반환:
        best_indices: list of int
            - 각 데이터별로 예측 중 코사인 유사도 평균이 가장 높은 예측의 인덱스를 담은 리스트
    """
    # 예측 및 정답 벡터의 노름(norm)을 계산합니다.
    pred_norms = np.linalg.norm(predictions, axis=2)  # shape: (N, p)
    gt_norms = np.linalg.norm(ground_truth, axis=2)     # shape: (N, t)
    
    # 배치별로 각 예측 벡터와 정답 벡터 간의 내적을 계산합니다.
    # np.einsum을 사용하여 (N, p, t) 형태의 결과를 얻습니다.
    dot_products = np.einsum('ipd,itd->ipt', predictions, ground_truth)  # shape: (N, p, t)
    
    # 예측과 정답 벡터의 노름을 외적하여 각 예측-정답 쌍에 대한 분모를 구성합니다.
    norms = pred_norms[:, :, None] * gt_norms[:, None, :]
    
    # 0으로 나누는 경우를 방지하기 위해 작은 값을 더해줍니다.
    epsilon = 1e-8
    cosine_similarities = dot_products / (norms + epsilon)
    
    # 각 예측 벡터에 대해 t개의 정답 벡터와의 코사인 유사도의 평균을 구합니다.
    avg_cosine_sim = np.mean(cosine_similarities, axis=2)  # shape: (N, p)
    
    # 각 데이터에서 평균 코사인 유사도가 가장 높은 예측의 인덱스를 찾습니다.
    best_indices = np.argmax(avg_cosine_sim, axis=1)
    
    return best_indices.tolist()

In [23]:
test_df['best_index'] = best_prediction_index(answer_ndarray, ground_truth_ndarray)

### best_index로 정답 문장 선택

In [24]:
def choose_answer(row):
    idx = row['best_index']
    return row[f'llm_answer_{idx+1}']

In [25]:
answers = test_df.apply(choose_answer, axis=1).tolist()

In [26]:
test_df

,id,description,correct_description_prompt,correct_description,category_exists,structuring_prompt,structured_json,structured_description,reason,rag_summary_prompt,llm_answer_1,llm_answer_2,llm_answer_3,best_index
0,0,타설작업 중 콘크리트펌프 관련 부딪힘 및 전도 사고 (펌프카 아웃트리거 바닥 고임목...,"당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고...",타설 작업 중 콘크리트 펌프 관련 부딪힘 및 전도 사고(펌프카 아웃트리거 바닥 고임...,Y,한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다...,"```json\n{\n ""발생 배경"": ""타설 작업 중 콘크리트 펌프 사용"",\n...","발생 배경: 타설 작업 중 콘크리트 펌프 사용, 사고 종류: 콘크리트 펌프 관련 부...","지반 침하 (아웃트리거 우측 상부 1개소), 아웃트리거 펼친 길이 상이, 타설 위치...",아래의 5개의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answ...,재발 방지 대책으로 작업 전 TBM 시 유사 및 동일 공종 작업자 안전교육을 실시하...,재발 방지 대책으로 작업 전 TBM 시 유사 및 동일 공종 작업자 안전교육을 실시하...,재발 방지 대책으로 작업 전 TBM 시 유사 및 동일 공종 작업자 안전교육 강화 및...,0
1,1,"절단작업 중 공구류 관련 절단, 베임 사고 (작업자의 불안전한 행동(숫돌 측면 사용...","당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고...","절단 작업 중 공구류 관련 절단, 베임 사고 (작업자의 불안전한 행동(숫돌 측면 사...",Y,한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다...,"```json\n{\n ""발생 배경"": ""절단 작업 중"",\n ""사고 종류"": ...","발생 배경: 절단 작업 중, 사고 종류: 공구류 관련 절단, 베임 사고, 사고 원인...",작업자의 불안전한 행동(숫돌 측면 사용) 및 보안면 미 착용,아래의 5개의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answ...,작업 전 안전교육 강화 및 작업장 위험요소 점검을 통한 재발 방지와 안전관리 교육 ...,"작업자 안전 교육 강화 및 보안면 착용 철저를 통한 재발 방지, 작업 전 안전 점검...","작업자 안전교육 강화 및 보안면 착용 의무 준수 철저요구, 작업 전 안전 점검 강화...",0
2,2,이동 중 떨어짐 사고 (작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛...,"당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고...",이동 중 떨어짐 사고 (작업자가 작업을 위해 이동 중 전방을 주시하지 않아 발을 헛...,Y,한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다...,"```json\n{\n ""발생 배경"": ""작업자가 작업을 위해 이동 중"",\n ...","발생 배경: 작업자가 작업을 위해 이동 중, 사고 종류: 이동 중 떨어짐 사고, 사...",작업자가 전방을 주시하지 않아 발을 헛디뎌 계단에서 굴러 넘어짐,아래의 5개의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answ...,이동 중 작업 시 전방 주시 습관 지도 및 계단 안전 수칙 교육 강화와 함께 해당 ...,이동 중 작업 시 전방 주시 및 안전 수칙 교육 강화와 계단 주변 안전 확보를 위한...,"이동 중 작업 시 전방 주시 및 안전 수칙 교육 강화, 계단 안전 점검 및 개선, ...",2
3,3,자재 관련 넘어짐(물체에 걸림) 사고 (작업 발판 위 벽돌 잔재를 밟고 넘어짐),"당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고...",자재 관련 넘어짐(물체에 걸림) 사고(작업 발판 위 벽돌 잔재를 밟고 넘어짐),Y,한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다...,"```json\n{\n ""발생 배경"": ""작업 발판 위 벽돌 잔재"",\n ""사고...","발생 배경: 작업 발판 위 벽돌 잔재, 사고 종류: 넘어짐 (물체에 걸림), 사고 ...",작업 발판 위 벽돌 잔재를 밟고 넘어짐,아래의 5개의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answ...,작업장 주변 정리정돈과 근로자 특별 안전교육 실시를 통한 향후 조치 계획.,작업장 주변 정리정돈 및 근로자 특별 안전교육 실시를 통한 향후 조치 계획.,작업장 주변 정리정돈 및 근로자 특별 안전교육 실시를 통한 향후 조치 계획.,0
4,4,해체작업 중 떨어짐 사고 (점심식사를 위한 이동시 작업자 부주의로 인한 추락사고 발생),"당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고...",해체작업 중 떨어짐 사고(점심 식사 시간 이동 중 작업자 부주의로 인한 추락 사고 발생),Y,한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다...,"```json\n{\n ""발생 배경"": ""해체 작업 중"",\n ""사고 종류"": ...","발생 배경: 해체 작업 중, 사고 종류: 떨어짐 사고 (점심 식사 시간 이동 중 작...",작업자 부주의,아래의 5개의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answ...,"점심 식사 시간 이동 시 안전 수칙 준수 교육 실시 및 안전모 착용 의무화, 작업 ...","점심 식사 시간 등 휴식 시간 중 작업 시 안전 수칙 준수 교육 실시, 작업 구역 ...","점심 식사 시간 등 휴식 시간 동안에는 작업 중단 및 안전 교육 실시, 작업복 착용...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,959,마감작업 중 건물 관련 부딪힘 사고 (약한 석재가 자연 파단하여 늑골 골절 됨),"당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고...",마감 작업 중 건물 관련 충돌 사고 (약한 석재가 자연 파단으로 늑골 골절됨),Y,한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다...,"```json\n{\n ""발생 배경"": ""마감 작업 중"",\n ""사고 종류"": ...","발생 배경: 마감 작업 중, 사고 종류: 건물 관련 충돌 사고, 사고 원인: 약한 ...",약한 석재의 자연 파단,아래의 5개의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answ...,재해 예방 안전교육 실시와 특이사항 없음.,재해 예방 안전교육 실시와 특이사항 없음.,재해 예방 안전교육 실시와 특이사항 없음.,0
960,960,조립작업 중 철망 관련 넘어짐 사고 (근로자가 작업 중 이동가능한 통로 확보 미흡),"당신은 맞춤법 수정 전문가입니다. 제공될 문장의 맞춤법을 수정하되, 형식을 유지하고...",조립작업 중 철망 관련 넘어짐 사고(근로자가 작업 중 이동 가능한 통로 확보 미흡),Y,한국 건설 공사 안전 사고 관련 문장을 언어 모델에 사용하기 위해 전처리하려 합니다...,"```json\n{\n ""발생 배경"": ""조립작업 중"",\n ""사고 종류"": ""...","발생 배경: 조립작업 중, 사고 종류: 철망 관련 넘어짐 사고, 사고 원인: 근로자...",근로자가 작업 중 이동 가능한 통로 확보 미흡,아래의 5개의 모범 답안 예시를 참고해 answer을 작성해주세요. 각각의 answ...,이동 통로 확보를 위한 작업 공간 정리 및 안전 표시판 설치를 통한 재발 방지 대책...,이동 통로 확보를 위한 작업 공간 정리 및 안전표지판 설치를 통한 재발 방지 대책 ...,작업 중 이동 가능한 통로 확보를 위한 안전표지판 설치 및 작업 전 안전 점검을 통...,1
961,961,타설작업 중 콘크리

### 결과 저장

In [27]:
test_df['answers'] = answers

In [28]:
test_df.to_csv('../data/test_08.csv', index=False)

### submit csv 생성

In [29]:
from sentence_transformers import SentenceTransformer

embedding_model_name = "jhgan/ko-sbert-sts"
embedding = SentenceTransformer(embedding_model_name)

# 문장 리스트를 입력하여 임베딩 생성
pred_embeddings = embedding.encode(answers)
print(pred_embeddings.shape)  # (샘플 개수, 768)

(964, 768)


In [30]:
submission = pd.read_csv('../output/sample_submission.csv', encoding = 'utf-8-sig')

# 최종 결과 저장
submission.iloc[:,1] = answers
submission.iloc[:,2:] = pred_embeddings
submission.head()

# 최종 결과를 CSV로 저장
submission.to_csv('../output/baseline_submission.csv', index=False, encoding='utf-8-sig')

In [31]:
answers

['재발 방지 대책으로 작업 전 TBM 시 유사 및 동일 공종 작업자 안전교육을 실시하고, 콘크리트 펌프카 점검 항목의 필수 점검을 포함한 수시 위험성 평가를 실시하며, 지반 상태 및 아웃트리거 펼침 길이 제한을 위한 사전 안전 교육을 철저히 시행하며, 한국전력공사 내 사고 사례 전파와 안전보건협의체, 안전시공보고회의, 합동안전점검 등을 통한 관리감독자 교육을 진행하는 계획.',
 '작업 전 안전교육 강화 및 작업장 위험요소 점검을 통한 재발 방지와 안전관리 교육 철저를 통한 향후 조치 계획.',
 '이동 중 작업 시 전방 주시 및 안전 수칙 교육 강화, 계단 안전 점검 및 개선, 작업자별 안전 수칙 준수 및 사고 발생 시 대처 요령 교육 실시 등 재발 방지 대책 마련.',
 '작업장 주변 정리정돈과 근로자 특별 안전교육 실시를 통한 향후 조치 계획.',
 '점심 식사 시간 이동 시 안전 수칙 준수 교육 실시 및 안전모 착용 의무화, 작업 구역 내 미끄럼 방지 조치 강화, 작업자 부주의 방지를 위한 주의 교육 실시 및 안전관리 책임자의 현장 감독 강화.',
 '작업 전 안전발판 설치 점검 및 작업절차 준수 교육 실시, 작업 구간 위험 요소 사전 파악 및 제거, 지속적인 안전점검 및 안전교육을 통한 동종 및 유사재해 방지.',
 '안전교육 강화 및 작업자 안전 수칙 준수 철저, 작업 전 주변 환경 점검 및 위험 요소 제거, 불안전한 행동 교정 및 작업 전 점검 철저, 수시 위험성 평가 실시 및 근로자 주지 및 확인.',
 'SPLICE PLATE 설치 시 안전거리 확보 및 작업 시 안전 교육 철저를 통한 재발 방지 대책 마련과 추가적으로 작업 환경 개선 및 안전모 착용 철저를 포함한 향후 조치 계획.',
 '작업발판 하단을 명확히 인지하도록 교육 실시 및 안전모 착용 점검 강화, 작업 시 집중력 향상을 위한 안전 수칙 준수 교육, 작업 전 위험 요소 사전 점검 및 미비 사항 시정 철저.',
 '향후 작업 시 주변 안전 위험 요소 제거와 안전 교육 철저.',
 '